<a href="https://colab.research.google.com/github/anihab/tokenization/blob/main/vocabulary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Libraries
import argparse
import gzip
import os

import pandas as pd
import numpy as np

!pip install Bio
!pip install tokenizers
from Bio import SeqIO
from tokenizers import Tokenizer, models, trainers, normalizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Train a BPE tokenizer on the full bacterial training directory to create the vocabulary file.

###Step 0

define vocabulary size and input data path
- input should be a path to a directory of bacterial fasta files

In [13]:
# Globals
VOCAB_SIZE = 4096

# Input
INPUT_PATH = "/ocean/projects/bio230026p/lindseyl/DATA/SEGMENTS/bacteria"

###Step 1

parse through all fasta files in full bacterial training directory
- to make input similar to input for pre-training, use sequences of at least 1500 nt (i.e., min sequence length = 1500)
- do not limit max sequence length

In [14]:
def parse_sequences(input_path):
  '''
  input: a directory of fasta files or a .txt file that has a list of fasta files
  output: a list of sequences
  '''
  sequences = [] # the only information we need is the sequences, we do not need to limit length
  if os.path.isdir(input_path):
    for filename in os.listdir(input_path):
      f = os.path.join(input_path, filename)
      if os.path.isfile(f):
        if f.endswith('.gz'):
          f = gzip.open(f, 'rt', encoding='utf-8')
        for record in SeqIO.parse(f, 'fasta'):
          seq = str(record.seq).upper() # min sequence length = 1500
          if len(seq) >= 1500:
            sequences.append(seq)
  return sequences

###Step 2

build vocabulary json - limit vocab size to 4096

In [15]:
def build_vocab(sequences):
  '''
  input: a list of sequences
  output: a vocabulary json file
  '''
  tokenizer = Tokenizer(models.BPE())
  # customize the tokenizer to handle DNA sequences
  tokenizer.normalizer = normalizers.Sequence([normalizers.NFKC()])
  # train the tokenizer on DNA sequences
  trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE)
  tokenizer.train_from_iterator(sequences, trainer=trainer)
  tokenizer.save("my_vocabulary.json")

###Step 3
build vocabulary on input

In [17]:
# build vocabulary
sequences = parse_sequences(INPUT_PATH)
build_vocab(sequences)